# 概要

本ファイルは、学習済みモデルを使用して、3D物体検出モデルの推論を実行します。 

> 本ファイルで実行するスクリプトはsecond.pytorchをベースとしています。

## Output
本ファイルを実行することで、`results`ディレクトリに、推論結果(result.pkl)を生成します。

```
└ second.pytorc
     └ checkpoints
          └ {model_dir}
                ├ results    <- 本ファイルにて生成されます
                   └ step_xxxx    
                       └─ result.pkl
```

# 準備

あらかじめ、 `data`ディレクトリに、KITTIデータセット、`velodyne_reduced`ディレクトリ、各メタ情報(pkl)ファイル、および、`second.pytorch/checkpoints/{model_dir}` ディレクトリに学習済みモデルがあることを確認してください。

> 本ファイルのデフォルトでは、公開の学習済みモデル`second.pytorch/checkpoints/pointpainting`を設定しています。


```
├data
   └ training
       ├── image_2
       ├── calib
       ├── label_2
       ├── velodyne
       └── velodyne_reduced     <-- 1-2_create_dataset_kitti.ipynb にて生成
   ├ kitti_gt_database            <-- 1-2_create_dataset_kitti.ipynb にて生成
   ├ kitti_infos_train.pkl        <-- 1-2_create_dataset_kitti.ipynb にて生成
   ├ kitti_infos_val.pkl          <-- 1-2_create_dataset_kitti.ipynb にて生成
   └ kitti_infos_trainval.pkl     <-- 1-2_create_dataset_kitti.ipynb にて生成

├second.pytorch
     └─ checkpoints
          └ {model_dir}
                ├── checkpoints.json
                ├── {model-step}.tckpt <-- 学習済みモデル
```

# 推論・評価

検証データ(val)に対して推論と評価を実施します。  
検証データの対象は`second.pytorch/second/data/ImageSets/val.txt`で定義されています。

In [1]:
# 設定
config_path = "../second.pytorch/second/configs/all.fhd_painted.config"

model_dir = "../second.pytorch/checkpoints/pointpainting"

In [2]:
!python ../second.pytorch/second/pytorch/train.py evaluate \
        --config_path=$config_path \
        --model_dir=$model_dir --measure_time=True --batch_size=4

[  41 1280 1056]
Restoring parameters from /work/second.pytorch/checkpoints/pointpainting/voxelnet-18570.tckpt
feature_map_size [1, 160, 132]
remain number of infos: 3769
Generate output labels...
[100.0%][===================>][3.50it/s][04:50>00:00]   
generate label finished(12.96/s). start eval:
avg example to torch time: 10.569 ms
avg prep time: 63.405 ms
avg voxel_feature_extractor time = 0.634 ms
avg middle forward time = 140.720 ms
avg rpn forward time = 75.411 ms
avg predict time = 17.016 ms
/usr/local/lib/python3.7/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 63 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.7/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 80 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.7/dist-packages/numba/cuda/dispatcher.py:488: NumbaP

## （オプション）推論結果の確認

推論データは pklファイルに保存されています。pklファイルを読み込みことで推論データを確認することができます。

In [3]:
import pickle

result_pkl = "../second.pytorch/checkpoints/pointpainting/eval_results/step_18570/result.pkl"

with open(result_pkl, 'rb') as f:
    result = pickle.load(f)  # FooクラスがないのでAttributeError例外

print("result size: ", len(result))
print(result[0]["box3d_lidar"])

result size:  3769
tensor([[46.2894, -4.6238,  0.0532,  0.4300,  1.6142,  1.6364,  1.1156]],
       device='cuda:0')


以上